#Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D,
Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import confusion_matrix, accuracy_score,
precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing import image

This section imports all the necessary libraries and modules required for building
and evaluating the deep learning model. It includes libraries for numerical computations
(numpy), data visualization (matplotlib), deep learning (TensorFlow and Keras), and evaluation
metrics (scikit-learn).

#Loading and Preprocessing the Dataset

In [ ]:
# Load Cifar-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Use a smaller subset of the data for quick testing
x_train_small, y_train_small = x_train[:1000], y_train[:1000]
x_test_small, y_test_small = x_test[:200], y_test[:200]
# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train_small = x_train_small.astype('float32') / 255.0
x_test_small = x_test_small.astype('float32') / 255.0
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
y_train_small = to_categorical(y_train_small, 10)
y_test_small = to_categorical(y_test_small, 10)

This section loads the CIFAR-10 dataset, which contains 60,000 32x32 color
images in 10 different classes. For quick testing, smaller subsets of the training and test data
are created. The images are normalized by scaling pixel values to the range [0, 1], and the
labels are converted to one-hot encoding format.

#Building the Model

In [ ]:
# Build the model function
def build_model(optimizer, loss):
model = Sequential([
Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
MaxPooling2D((2, 2)),
Conv2D(64, (3, 3), activation='relu'),
MaxPooling2D((2, 2)),
Conv2D(128, (3, 3), activation='relu'),
Flatten(),
Dense(512, activation='relu'),
Dropout(0.5),
Dense(10, activation='softmax')
])
model.compile(optimizer=optimizer, loss=loss,
metrics=['accuracy'])
return model


This section defines a function build_model that constructs a Convolutional
Neural Network (CNN) using the Keras Sequential API. The model consists of convolutional
layers, max-pooling layers, a dense layer with dropout for regularization, and a final output layer
with softmax activation. The model is compiled with the specified optimizer and loss function.

#Model Training and Evaluation

In [ ]:
# Initialize optimizers and loss functions
optimizers = ['adam', 'sgd', 'rmsprop']
loss_functions = ['categorical_crossentropy', 'mean_squared_error']

# Clear previous history from memory to avoid confusion
tf.keras.backend.clear_session()

# Function to evaluate the model and compute metrics
def evaluate_model(model, x_test, y_test):
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)
conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
specificity = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
return conf_matrix, accuracy, precision, recall, f1, specificity
history_list = []
metrics_list = []

for opt_name in optimizers:
for loss_name in loss_functions:
print(f"\nTraining with Optimizer: {opt_name}, Loss Function: {loss_name}\n")

if opt_name == 'adam':
optimizer = tf.keras.optimizers.legacy.Adam()
elif opt_name == 'sgd':
optimizer = tf.keras.optimizers.legacy.SGD()
elif opt_name == 'rmsprop':
optimizer = tf.keras.optimizers.legacy.RMSprop()
model = build_model(optimizer, loss_name)

history = model.fit(x_train_small, y_train_small, epochs=2,
validation_data=(x_test_small, y_test_small), batch_size=64,
verbose=2)
history_list.append((opt_name, loss_name, history))

# Evaluate model and collect metrics
conf_matrix, accuracy, precision, recall, f1, specificity =
evaluate_model(model, x_test_small, y_test_small)
metrics_list.append((opt_name, loss_name, conf_matrix,
accuracy, precision, recall, f1, specificity))

# Clear session to avoid conflicts in subsequent trainings
tf.keras.backend.clear_session()

This section initializes a list of optimizers and loss functions to experiment with. It
defines a function `evaluate_model` to compute and return various evaluation metrics. The
models are trained and evaluated using different combinations of optimizers and loss functions.
The metrics for each combination are stored in `metrics_list`.

# Displaying Metrics and Saving the Best Model

In [ ]:
# Displaying metrics
for opt_name, loss_name, conf_matrix, accuracy, precision, recall, f1,
specificity in metrics_list:
print(f"Optimizer: {opt_name}, Loss Function: {loss_name}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Specificity: {specificity}")
print("\n")

# Save the best model's weights for testing
model.save_weights('best_model_weights.h5')

This section displays the evaluation metrics for each combination of optimizer and
loss function. It also saves the weights of the best-performing model for later use.


#Custom Image Prediction

In [ ]:
# Function to preprocess the image
def load_and_preprocess_image(img_path):
img = image.load_img(img_path, target_size=(32, 32))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
return img_array

# Load the best model
best_model = build_model(tf.keras.optimizers.legacy.Adam(),
'categorical_crossentropy')
best_model.load_weights('best_model_weights.h5')

# Load your custom image
img_path = '/content/bird.jpg' # Replace with your image path
img_array = load_and_preprocess_image(img_path)

# Predict the class of the custom image
prediction = best_model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map class index to class name
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
predicted_object = class_names[predicted_class]
print(f"Predicted object: {predicted_object}")


This section defines a function to preprocess custom images for prediction. It then
loads the best model, preprocesses a custom image, and predicts its class. The predicted class
index is mapped to the corresponding class name and printed.